In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 73.9 MB/s 
     |████████████████████████████████| 77 kB 9.4 MB/s 
     |████████████████████████████████| 880 kB 84.0 MB/s 
     |████████████████████████████████| 596 kB 73.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c7dbc2e8855f5d8c1b9b60cf6ee7951747dfe0e53c58babe860c411e8a557d1c
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/Shareddrives/DL

Mounted at /content/gdrive/
/content/gdrive/Shareddrives/DL


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertTokenizer
from transformers import PLBartTokenizer, PLBartForConditionalGeneration
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
plb_model =  PLBartForConditionalGeneration.from_pretrained("PLBART Model",max_position_embeddings=1500,ignore_mismatched_sizes=True).to(device)
plb_model.to(device)

PLBartForConditionalGeneration(
  (model): PLBartModel(
    (shared): Embedding(50005, 768, padding_idx=1)
    (encoder): PLBartEncoder(
      (embed_tokens): Embedding(50005, 768, padding_idx=1)
      (embed_positions): PLBartLearnedPositionalEmbedding(1502, 768)
      (layers): ModuleList(
        (0): PLBartEncoderLayer(
          (self_attn): PLBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,)

In [5]:
tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-base",src_lang="java", tgt_lang="java")

def PLBartTokenizerForSeq2SeqTest(x):
  return tokenizer.batch_encode_plus(x,max_length=750,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

Downloading:   0%|          | 0.00/963k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783 [00:00<?, ?B/s]

In [70]:
tc2code = pd.read_json('./data.jsonl',lines=True)

In [71]:
tc2code.head()

,testCode,sourceCode
0,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...
1,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Use...
2,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.util.regex....
3,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.awt.event.C...
4,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport javax.swing.*;\n...


In [72]:
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(tc2code['testCode'], tc2code['sourceCode'], test_size=0.2, shuffle=False)
_, X_test, _, y_test = train_test_split(X_test, y_test, test_size=0.1, shuffle=False)

In [74]:
print(len(X_test.to_list()))

459


In [75]:
print(type(PLBartTokenizerForSeq2SeqTest(X_test.to_list()[1:2])))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [76]:
print(y_test.to_list()[0])

/*
 * Created on 14/mar/2010
 * Copyright (C) 2010 by Andrea Vacondio.
 *
 * This program is free software; you can redistribute it and/or modify it under the terms of the 
 * GNU General Public License as published by the Free Software Foundation; 
 * either version 2 of the License.
 * This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
 * without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
 * See the GNU General Public License for more details.
 * You should have received a copy of the GNU General Public License along with this program; 
 * if not, write to the Free Software Foundation, Inc., 
 *  59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
 */
package org.pdfsam.guiclient.business.listeners;

import java.awt.event.InputEvent;
import java.awt.event.MouseWheelEvent;
import java.awt.event.MouseWheelListener;

import org.pdfsam.guiclient.gui.components.JPreviewImage;

/**
 * Listen for the mouse whe

In [77]:
from torch.utils import data
class CodeTCDatasetTest(data.Dataset):
    def __init__(self, testTensor,expectedSourceCode,testCodeInput):
      self.testTensor=testTensor
      self.expectedSourceCode=expectedSourceCode
      self.testCodeInput=testCodeInput


    def __getitem__(self, index):
        return self.testTensor[index],self.expectedSourceCode[index],self.testCodeInput[index]


    def __len__(self):
        return len(self.testCodeInput)


def collate_fn(data):
    testTensorList,expectedSourceCode,testCodeInput = zip(*data)

    testTensors = torch.stack(testTensorList, 0)
      
    return testTensors, expectedSourceCode, testCodeInput

def get_test_loader(testTensor,expectedSourceCode,testCodeInput,batch_size):
    code2tctest = CodeTCDatasetTest(testTensor,expectedSourceCode,testCodeInput)
    data_loader = torch.utils.data.DataLoader(dataset=code2tctest, 
                                              batch_size=batch_size,
                                              collate_fn=collate_fn)
    return data_loader

In [78]:
X_test_tensors=PLBartTokenizerForSeq2SeqTest(X_test.to_list())
input_ids_test = X_test_tensors['input_ids']

In [80]:
print(len(input_ids_test))

459


In [100]:
batch_size=32
test_loader=get_test_loader(input_ids_test,y_test.to_list(),X_test.to_list(),batch_size)

In [101]:
print(type(X_test_tensors))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [102]:
def getPredictionsGreedy(inputs,model,device):
      greedy_output = model.generate(inputs.to(device), max_length=1500)
      sequences=[]
      for i in range(greedy_output.shape[0]):
        sequences.append(tokenizer.decode(greedy_output[i], skip_special_tokens=True))
      return sequences

In [103]:
def getPredictedTexts(dataloader_test,genAlgo,model,device,targIters=1,need_targ_iters=False):
    predictions=[]
    allExpectedprogs=[]
    allIpProgs=[]
    idx=0
    with torch.no_grad():
      for ip,estr,istr in dataloader_test:
          if idx>=targIters and need_targ_iters:
            break
          print(f"iter{idx} of {len(dataloader_test)}")
          prediction=genAlgo(ip,model,device)
          predictions=predictions+prediction
          expected=[x for x in estr]
          inputs=[x for x in istr]
          allExpectedprogs=allExpectedprogs+expected
          allIpProgs=allIpProgs+inputs
          idx+=1
    return allExpectedprogs,predictions,allIpProgs

In [104]:
allExpectedprogsGreedy,predictionsGreedy,ipTestCode=getPredictedTexts(test_loader,getPredictionsGreedy,plb_model,device)

iter0 of 15
iter1 of 15
iter2 of 15
iter3 of 15
iter4 of 15
iter5 of 15
iter6 of 15
iter7 of 15
iter8 of 15
iter9 of 15
iter10 of 15
iter11 of 15
iter12 of 15
iter13 of 15
iter14 of 15


In [99]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [106]:
print(allExpectedprogsGreedy[12])

/*
 * Created on 06-Sep-2009
 * Copyright (C) 2009 by Andrea Vacondio.
 *
 * This program is free software; you can redistribute it and/or modify it under the terms of the 
 * GNU General Public License as published by the Free Software Foundation; 
 * either version 2 of the License.
 * This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
 * without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
 * See the GNU General Public License for more details.
 * You should have received a copy of the GNU General Public License along with this program; 
 * if not, write to the Free Software Foundation, Inc., 
 *  59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
 */
package org.pdfsam.guiclient.business.thumbnails.callables;

import java.awt.image.BufferedImage;
import java.util.concurrent.Callable;

import org.apache.log4j.Logger;
import org.icepdf.core.pobjects.Document;
import org.icepdf.core.pobjects.PDimension;
i

In [105]:
print(ipTestCode[12])

/*
 * This file was automatically generated by EvoSuite
 */

package org.pdfsam.guiclient.business.thumbnails.callables;

import org.junit.Test;
import org.junit.runner.RunWith;
import org.evosuite.junit.EvoSuiteRunner;
import static org.junit.Assert.*;
import org.icepdf.core.pobjects.Document;
import org.junit.BeforeClass;
import org.pdfsam.guiclient.business.thumbnails.callables.IcePdfThumbnailCallable;
import org.pdfsam.guiclient.commons.panels.JVisualPdfPageSelectionPanel;
import org.pdfsam.guiclient.dto.VisualPageListItem;

@RunWith(EvoSuiteRunner.class)
public class IcePdfThumbnailCallableEvoSuiteTest {

  @BeforeClass 
  public static void initEvoSuiteFramework(){ 
    org.evosuite.Properties.REPLACE_CALLS = true; 
  } 


  @Test
  public void test0()  throws Throwable  {
      Document document0 = new Document();
      IcePdfThumbnailCallable icePdfThumbnailCallable0 = new IcePdfThumbnailCallable(document0, (VisualPageListItem) null, (JVisualPdfPageSelectionPanel) null, (-1L));

In [107]:
print(predictionsGreedy[12])

.pdpdfsam.guiclient.business.thumbnails.callables; import org.apache.commons.lang.StringUtils; import org.pdfsam.guiclient.core.dnd.PdfElement; import org.pdfsam.guiclient.core.dnd.PdfRenderingHints; import org.pdfsam.guiclient.core.dnd.PdfRenderingHints; import org.pdfsam.guiclient.core.dnd.PdfTag; import org.pdfsam.guiclient.core.dnd.PdfTagWithParameters; import org.pdfsam.guiclient.core.dnd.PdfSource; import org.pdfsam.guiclient.core.dnd.PdfRenderingHintsProvider; import.util.ArrayList; /** * @author Philip * */ public class IcePdfThumbnailCallable extends PdfThumbnailCallable{ private final String MY_CALL_METHOD = "call"; private final String MY_CALL_METHOD_NAME = "call_method"; private final PdfVisual page; public IcePdfThumbnailCallable(String function, PdfVisual page, long rct) { super(function, page, rct); this.page = page; } @Override public void call() { try { page.waitFor(); } catch(InterruptedException e) { e.printStackTrace(); } catch (Exception e) { e.printStackTrace(); }

In [111]:
print(predictionsGreedy[52])

.pdpdpdfsam.guiclient.updates; import.io.BufferedReader; import.io.InputStreamReader; import.io.InputStreamWriter; import.util.ArrayList; import.util.HashMap; import.util.Iterator; import.util.List; import.util.Map; import org.apache.commons.cli.CommandLine; import org.apache.commons.cli.ParseException; import org.apache.commons.cli.PosixParser; import org.apache.commons.cli.PosixParserFactory; import org.apache.commons.cli.PosixParser; import org.apache.commons.cli.Options; import org.apache.commons.cli.ParseException; import org.apache.commons.cli.PosixParserFactory; import org.apache.commons.cli.PosixParser; import org.apache.commons.cli.Options; import org.apache.commons.cli.ParseException; import org.apache.commons.cli.PosixParserFactory; import org.apache.log4j.Logger; import org.pdfsam.guiclient.updates.Update; import org.pdfsam.guiclient.updates.UpdateException; import org.pdfsam.guiclient.updates.UpdateManager; import org.pdfsam.guiclient.updates.UpdateManager.UpdateListener; 

In [108]:
predictiondf_plb = pd.DataFrame(list(zip(allExpectedprogsGreedy, predictionsGreedy,ipTestCode)), columns =['Expected', 'predictions','ipUnitTest']) 

In [109]:
predictiondf_plb.to_json('ResultValidation/predictions_plbart.jsonl',orient='records', lines=True)